In [1]:
using PyPlot

In [2]:
alpha(r::Float64,k::Float64,l::Float64) = acos((r^2.0 + k^2.0 - l^2.0)/(2.0*k*r))    

alpha (generic function with 1 method)

In [3]:
beta(r::Float64,k::Float64,l::Float64) = acos((r^2.0 + l^2.0 - k^2.0)/(2.0*l*r))  

beta (generic function with 1 method)

In [4]:
function r(tau::Array{Float64},b::Float64,k::Float64,l::Float64,T::Float64)
    v = 2.0*sqrt((k+l)^2.0-b^2.0)/T
    return sqrt(b^2.0 .+ (v.*tau).^2.0)
end

r (generic function with 1 method)

In [5]:
tau(t::Array{Float64},P::Float64,t_n::Float64) = mod(t.-t_n+(P/2),P)-(P/2.0)

tau (generic function with 1 method)

In [6]:
function overlap(rs::Array{Float64},k::Float64,l::Float64)
    result = Float64[]
    for r in rs
        if r >= l+k
            push!(result,0.0)
        elseif r <= l-k
            push!(result,pi*k^2.0)
        else
            push!(result,(k^2.0)*(alpha(r,k,l)-cos(alpha(r,k,l))*sin(alpha(r,k,l)))+(l^2.0)*(beta(r,k,l)-cos(beta(r,k,l))*sin(beta(r,k,l))))
        end
    end
    return result
end

overlap (generic function with 1 method)

In [7]:
function transit(ts::Array{Float64},F_0::Float64,k::Float64,l::Float64,P::Float64,T::Float64,t_n::Float64,b::Float64)
    taus = tau(ts,P,t_n)
    rs = r(taus,b,k,l,T)
    overlaps = overlap(rs,k,l)
    return F_0*(1.0.-overlaps./(pi*l^2.0))
end

transit (generic function with 1 method)

In [18]:
function phase_folder(x,y)
    #phase-folds data
end

function plot_transit(x_data,y_data,fit)
    #Outputs a phase folded normalized plot of light curve with best fit
    
    #Well currently it doesn't, but it will!
end

function density(fit)
    #given the fit, calculate the density of the star (based on the duration of transit)
end

density (generic function with 1 method)